Swap a voice in the audio fragment with a computer generated one

At its code the doemo does:
- ASR of what is said in the file. E.g converting audio to text
- Adding punctuation and capitalization in the text
- Generating spectrogram from resulting text
- Generating waveform audio from the spectrogram

In [1]:
# Ignore pre-production warnings
import warnings
warnings.filterwarnings('ignore')
import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts
import IPython

[NeMo W 2023-03-21 11:28:01 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-03-21 11:28:03 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-03-21 11:28:04 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-03-21 11:28:04 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-03-21 11:28:04 experimental:27] Module <class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokeni

In [2]:
# Download audio sample which we'll try
# This is a sample from LibriSpeech Dev Clean dataset - the model hasn't seen it before
Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# Listen to it
IPython.display.Audio(Audio_sample)

--2023-03-21 11:28:07--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.108.2
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.108.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav.1’

2086-149220-0033.wa 100%[===================>] 232.39K   322KB/s    in 0.7s    

2023-03-21 11:28:09 (322 KB/s) - ‘2086-149220-0033.wav.1’ saved [237964/237964]



Instantiate pre-trained NeMo models which we'll use

`from_pretrained(...)` API downloads and initialized model directly from the cloud


In [7]:
# Speech recognition model - Quartznet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5").cuda()

# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='punctuation_en_distilbert').cuda()

# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.FastPitchModel.from_pretrained(model_name="tts_en_fastpitch").cuda()

# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.HifiGanModel.from_pretrained(model_name="tts_hifigan").cuda()

[NeMo I 2023-03-21 11:44:37 cloud:56] Found existing object /home/mat/.cache/torch/NeMo/NeMo_1.16.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2023-03-21 11:44:37 cloud:62] Re-using file from: /home/mat/.cache/torch/NeMo/NeMo_1.16.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2023-03-21 11:44:37 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-03-21 11:44:38 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2023-03-21 11:44:38 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2023-03-21 11:44:38 features:286] PADDING: 16
[NeMo I 2023-03-21 11:44:38 save_restore_connector:247] Model EncDecCTCModel was successfully restored from /home/mat/.cache/torch/NeMo/NeMo_1.16.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2023-03-21 11:44:38 cloud:56] Found existing object /home/mat/.cache/torch/NeMo/NeMo_1.16.0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo.
[NeMo I 2023-03-21 11:44:38 cloud:62] Re-using file from: /home/mat/.cache/torch/NeMo/NeMo_1.16.0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo
[NeMo I 2023-03-21 11:44:38 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-03-21 11:44:41 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmplljsgm6b/tokenizer.vocab_file, merges_files: None, special_tokens_dict: {}, and use_fast: Fals

Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-03-21 11:44:42 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2023-03-21 11:44:42 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_audio: false
    audio_file: null
    sample_rate: 16000
    use_bucketing: true
    batch_size: 32
    preload_audios: true
    use_tarred_dataset: false
    label_info_save_dir: null
    text_file: text_train.txt
    labels_file: labels_train.txt
    tokens_in_batch: null
    max_seq_length: 128
    num_samples: -1
    use_cache: true
    cache_dir: null
    get_label_frequences: false
    verbose: true
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    shard_strategy: scatter
    shuffle: true

[NeMo I 2023-03-21 11:44:46 save_restore_connector:247] Model PunctuationCapitalizationModel was successfully restored from /home/mat/.cache/torch/NeMo/NeMo_1.16.0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo.
[NeMo I 2023-03-21 11:44:46 cloud:56] Found existing object /home/mat/.cache/torch/NeMo/NeMo_1.16.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2023-03-21 11:44:46 cloud:62] Re-using file from: /home/mat/.cache/torch/NeMo/NeMo_1.16.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo
[NeMo I 2023-03-21 11:44:46 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-03-21 11:44:49 tokenize_and_classify:87] Creating ClassifyFst grammars.


...

[NeMo I 2023-03-21 11:45:23 features:286] PADDING: 1
[NeMo I 2023-03-21 11:45:23 save_restore_connector:247] Model FastPitchModel was successfully restored from /home/mat/.cache/torch/NeMo/NeMo_1.16.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2023-03-21 11:45:23 cloud:56] Found existing object /home/mat/.cache/torch/NeMo/NeMo_1.16.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2023-03-21 11:45:23 cloud:62] Re-using file from: /home/mat/.cache/torch/NeMo/NeMo_1.16.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2023-03-21 11:45:23 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-03-21 11:45:27 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2023-03-21 11:45:27 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2023-03-21 11:45:27 features:286] PADDING: 0


[NeMo W 2023-03-21 11:45:27 features:263] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2023-03-21 11:45:27 features:286] PADDING: 0
[NeMo I 2023-03-21 11:45:28 save_restore_connector:247] Model HifiGanModel was successfully restored from /home/mat/.cache/torch/NeMo/NeMo_1.16.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [6]:
# convert audio sample to text
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
    raw_text = transcription

# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]
print(f'\nRaw recognized text: {raw_text}. \nText with capitalization and punctuation: {text}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo I 2023-03-21 11:44:33 punctuation_capitalization_model:1153] Using batch size 1 for inference
[NeMo I 2023-03-21 11:44:33 punctuation_capitalization_infer_dataset:127] Max length: 28
[NeMo I 2023-03-21 11:44:33 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-03-21 11:44:33 data_preprocessing:406] Min: 26 |                  Max: 26 |                  Mean: 26.0 |                  Median: 26.0
[NeMo I 2023-03-21 11:44:33 data_preprocessing:412] 75 percentile: 26.00
[NeMo I 2023-03-21 11:44:33 data_preprocessing:413] 99 percentile: 26.00


100%|██████████| 1/1 [00:00<00:00, 56.64batch/s]


Raw recognized text: well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very likt the old portrait. 
Text with capitalization and punctuation: Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very likt the old portrait.


In [8]:
# A helper function which combines TTS models to go directly from 
# text to audio
def text_to_audio(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').detach().numpy()

In [9]:
# This is our original audio sample
IPython.display.Audio(Audio_sample)

In [10]:
# This is what was recognized by the ASR model
print(raw_text)

well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very likt the old portrait


In [11]:
# This is how punctuation model changed it
print(text)

Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very likt the old portrait.


In [13]:
# Without punctuation
IPython.display.Audio(text_to_audio(raw_text), rate=22050)

[NeMo W 2023-03-21 11:46:14 fastpitch:262] parse() is meant to be called in eval mode.
[NeMo W 2023-03-21 11:46:14 fastpitch:327] generate_spectrogram() is meant to be called in eval mode.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.82 GiB total capacity; 2.68 GiB already allocated; 10.62 MiB free; 2.71 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Final result - with punctuation
IPython.display.Audio(text_to_audio(text), rate=22050)